* add for SMA and other data
* add the button
* add so that the different province data can be in 1 workbook but in different spreadsheet
* make it so that the data's columns are just like what's in the web (for this, will have to understand the th and td)

# Import Library

In [1]:
import selenium
import csv
from selenium import webdriver
from scrapy.selector import Selector

In [2]:
import csv
from bs4 import BeautifulSoup as bs
from time import sleep
import pandas as pd
import numpy as np

In [3]:
from selenium import webdriver  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.chrome.options import Options

get whole raw data

In [4]:
def get_raw_data(url_str):
    driver = webdriver.Chrome('/Users/flo_anggi/Documents/going_headless/chromedriver')
    driver.get(url_str)
    
    while True:
        try :
            driver.find_element_by_class_name("btn-tab.btn-dist").click()
            break
        except:
            pass
    
    html_str = driver.find_elements_by_id("myTableData")[1].get_attribute("outerHTML")
    soup = bs(html_str.encode(),'html.parser')
    
    return soup


get whole raw data into correct data types understood by english excel

In [5]:
def convert_num_with_dots(num):
    if num == '-':
        return np.nan
    elif '.' in num:
        return float(num)*1000
    else:
        return int(num)

def convert_num_with_commas(num):
    if num == '-':
        return np.nan
    elif ',' in num:
        return float(num.replace(',','.'))
    else:
        return int(num)

def get_clean_data(soup):
    index = []
    df = pd.DataFrame({})
    i = 0

    for tr in soup.find_all("tr")[3:]:
        data = []
        index.append(tr.find_all("td")[0].text.strip())
        for td in tr.find_all("td")[1:]:
            data.append(td.text.strip())
        df[index[i]] = data
        i = i+1
        
    arrays = [['Bahasa Indonesia','Bahasa Indonesia','Bahasa Inggris','Bahasa Inggris'
            ,'Matematika','Matematika','IPA','IPA'
            ,'Semua Mata Pelajaran','Semua Mata Pelajaran']
           ,['Jumlah','%','Jumlah','%','Jumlah','%','Jumlah','%','Jumlah','%']]
    
    df_clean = df.set_index(pd.MultiIndex.from_arrays(arrays)).transpose()
    
    list_cols = ['Bahasa Indonesia','Bahasa Inggris','Matematika','IPA','Semua Mata Pelajaran']
    dot_col = 'Jumlah'
    comma_col = '%'

    for col in list_cols:
        df_clean.loc[:, (col,comma_col)] = df_clean.loc[:, (col,comma_col)].apply(convert_num_with_commas)
        df_clean.loc[:, (col,dot_col)] = df_clean.loc[:, (col,dot_col)].apply(convert_num_with_dots)
    
    return df_clean

get della's request:
* group the ones with <=55 and the ones with >55
* get only maths and bahasa indonesia

In [6]:
def get_filtered_data(data,score_range,lessons):
    list_col = []
    for lesson in lessons:
        list_col.append((lesson,'Jumlah'))
        list_col.append((lesson,'%'))

    score_range_idx = list(data.index).index(score_range)
    score_range_bottom = list(data.index)[score_range_idx+1]

    data_fin = pd.DataFrame(columns=list_col,index=[score_range,score_range_bottom])
    data_fin.columns = pd.MultiIndex.from_tuples(data_fin.columns)
    
    for lesson in lessons:
        sum_all = data[lesson]['Jumlah'].sum()
        sum_upper = data[:score_range][lesson]['Jumlah'].sum()
        sum_bottom = data[score_range:].drop(index=score_range)[lesson]['Jumlah'].sum()

        data_fin.loc[score_range, (lesson,'Jumlah')] = sum_upper
        data_fin.loc[score_range, (lesson,'%')] = sum_upper/sum_all

        data_fin.loc[score_range_bottom, (lesson,'Jumlah')] = sum_bottom
        data_fin.loc[score_range_bottom, (lesson,'%')] = sum_bottom/sum_all
        
        index_upper = '0.00-'+score_range.split('-')[1]
        index_bottom = score_range_bottom.split('-')[0]+'-60.00'
        
    return data_fin.set_index(pd.Index([index_upper,index_bottom]))

write it all into multiple spreadsheets in one workbook

In [7]:
# education level: smp, sma, smk, paketb, paketc

def get_url(province_code,education_level):
    if province_code < 10:
        url_str = 'https://hasilun.puspendik.kemdikbud.go.id/#2019!'+education_level+'!capaian_wilayah!0'+str(province_code)+'&99&999!T&T&T&T&1&!2!&'
    elif province_code >= 10:
        url_str = 'https://hasilun.puspendik.kemdikbud.go.id/#2019!'+education_level+'!capaian_wilayah!'+str(province_code)+'&99&999!T&T&T&T&1&!2!&'
    return url_str
        

In [8]:
"""
Cara pakai:
* ganti education_level dengan: 'smp', 'sma', 'smk', 'paketb', 'paketc'
* ganti score_range dengan nilai apapun yg ada di rentang nilai seperti di link, misal '10.01-15.00'
* tambah dan kurangi lessons dengan apapun seperti 'Bahasa Indonesia', 'Matematika', 'Bahasa Inggris', 'IPA', 'Semua Mata Pelajaran'
* ganti rentang province_code dengan angka apapun antara 1 sampai 34

"""

education_level = 'smp'
score_range = '50.01-55.00'
lessons = ['Bahasa Indonesia','Matematika']

writer = pd.ExcelWriter('della_all_nilai_smp.xlsx', engine='xlsxwriter')

for province_code in range(1,35):
# for province_code in range(1,4):
    url_str = get_url(province_code,education_level)
    soup = get_raw_data(url_str)
    data = get_clean_data(soup)
    filter_data = get_filtered_data(data,score_range,lessons)
    filter_data.to_excel(writer,sheet_name = 'provinsi_'+str(province_code))
#     data.to_excel(writer,sheet_name = 'provinsi_'+str(province_code))

writer.save()